<span style="font-size:36px"><b>Preprocess Language Model Corpus</b></span>

Copyright &copy; 2020 Gunawan Lumban Gaol

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language overning permissions and limitations under the License.

# Import Packages

In [1]:
import os
import re
import functools
import string
import random

import pandas as pd
import num2words
from tqdm import tqdm_notebook as tqdm

from gurih.models.utils import sparsity

In [2]:
def print_stats(stats):
    print(f"Number of documents: {stats['n']}")
    print(f"Sum of Ni: {stats['Ni']}")
    print(f"Vocab size: {len(stats['V'])}")
    print(f"Sparsity: {stats['Sd']}")

# RegExp Collections

In [3]:
num_dict = {
    "0": "kosong",
    "1": "satu",
    "2": "dua",
    "3": "tiga",
    "4": "empat",
    "5": "lima",
    "6": "enam",
    "7": "tujuh",
    "8": "delapan",
    "9": "sembilan",
    "*": "",
    "x": "",
    " ": "",
    "-": ""
}

simplified_punctuation_map = {"?": ".",
                              "!": ".",
                              ";": ",",
                              "'": "",
                              "-": " "}

In [4]:
# 4,69% --> empat koma enam sembilan persen
repl_percentage_with_comma = lambda x: " ".join([num2words.num2words(int(num), lang='id') for num in x.group(1)]) + " koma " + " ".join([num2words.num2words(int(num), lang='id') for num in x.group(2)]) + " persen"
regex_percentage_with_comma = functools.partial(re.sub, "(\d+),(\d+)[\s]{0,1}%", repl_percentage_with_comma)

# rp400.000 --> 400.000
regex_currency = functools.partial(re.sub, "(rp)([\s]{0,1}\d)", r"\2")

# 400.000/bulan --> 400.000 per bulan
regex_per_month = functools.partial(re.sub, "(/)([\s]{0,2}bulan)", r" per \2")

# 3x24 jam --> 3 kali 24 jam
# 3x --> 3 kali 
# 3 x 24 jam --> 3 kali 24 jam
regex_frequency = functools.partial(re.sub, "(\d+)\s?x\s?(\d+)?", r"\1 kali \2")

# kontrak 380 170**xx15 96 --> kontrak tiga delapan kosong  satu tujuh kosong     satu lima  sembilan enam
regex_contract_number = functools.partial(re.sub, "(kontrak[\s]{0,2})([\d\s\*x]+)", lambda x: f"{x.group(1)}{' '.join([num_dict[num] for num in x.group(2)])}")

#  021-238432 --> kosong dua satu  dua tiga delapan empat tiga dua
regex_phone_number = functools.partial(re.sub, "(\s)(0[\d\s\*x-]+)", lambda x: f"{x.group(1)}{' '.join([num_dict[num] for num in x.group(2)])}")

# @hoewwedr --> ''
regex_remove_username = functools.partial(re.sub, "(@[A-Za-z0-9]+)", '')

# #yangkamumau --> ''
regex_remove_hashtag = functools.partial(re.sub, "(#[A-Za-z0-9]+)", '')

# https://api.whatsapp.com/send=?phone18273 --> ''
regex_remove_url = functools.partial(re.sub, "^https?:\/\/.*[\r\n]*", '')

# awdaw.....,,,, --> awdaw.,
regex_multiple_punctuations = functools.partial(re.sub, r'([.,])\1+', r'\1')

# gitu.aku sebel ,banget sama dia , dan kamu . --> gitu. aku sebel, banget sama dia, dan kamu. 
regex_triplet_punctuations = functools.partial(re.sub, r'(.)\s?(\.|,)\s?(.)?', r'\1\2 \3')

# <see simplified punctuation dictionary>
regex_simplify_punctuations = lambda x: ''.join([simplified_punctuation_map.get(c, c) for c in x])

# remove non used character map
regex_remove_non_charmap = functools.partial(re.sub, '[^A-Za-z0-9., ]', '')

# 200.000 -->  dua ratus ribu 
# 200000 --> dua ratus ribu
repl_spell_number = lambda x: ' ' + num2words.num2words(int(re.sub(r'[.,]', '', x.group(1))), lang='id') + ' '
regex_spell_number = functools.partial(re.sub, r'((\d+[.,]?)+)', repl_spell_number)

# bunga / buah --> bunga atau buah
# bunga/ buah --> bunga atau buah
# bunga /buah --> bunga atau buah
# bunga/buah --> bunga atau buah
regex_remove_multiple_spaces = functools.partial(re.sub, ' +', ' ')
regex_atau = functools.partial(re.sub, r'(. ?)\/( ?.)', lambda x: regex_remove_multiple_spaces(x.group(1) + " atau " + x.group(2)))

In [5]:
assert regex_percentage_with_comma("4,69%") == "empat koma enam sembilan persen"
assert regex_currency("rp400.000") == "400.000"
assert regex_per_month("400.000/bulan") == "400.000 per bulan"
assert regex_frequency("3x24 jam") == "3 kali 24 jam"
assert regex_frequency("3x") == "3 kali "
assert regex_frequency("3 x 24 jam") == "3 kali 24 jam"
assert regex_contract_number("kontrak 380 170**xx15 96") == "kontrak tiga delapan kosong  satu tujuh kosong     satu lima  sembilan enam"
assert regex_phone_number(" 021-238432") == " kosong dua satu  dua tiga delapan empat tiga dua"
assert regex_remove_username("@hoewwedr") == ''
assert regex_remove_hashtag("#yangkamumau") == ''
assert regex_remove_url("https://api.whatsapp.com/send=?phone18273") == ''
assert regex_multiple_punctuations("awdaw.....,,,,") == "awdaw.,"
assert regex_triplet_punctuations("gitu.aku sebel ,banget sama dia , dan kamu .") == "gitu. aku sebel, banget sama dia, dan kamu. "
assert regex_simplify_punctuations("?!;'-") == ".., "
assert regex_spell_number("200000") == ' dua ratus ribu '
assert regex_spell_number("200.000") == ' dua ratus ribu '
assert regex_spell_number("200000000") == ' dua ratus juta '
assert regex_spell_number("200,000,000") == ' dua ratus juta '
assert regex_atau("bunga / buah") == "bunga atau buah"
assert regex_atau("bunga/ buah") == "bunga atau buah"
assert regex_atau("bunga /buah") == "bunga atau buah"
assert regex_atau("bunga/buah") == "bunga atau buah"

# Mediakonsumen Corpus

Scrape text from https://mediakonsumen.com/tag/home-credit-indonesia.

In [6]:
with open("../../dataset/corpus/raw/mediakonsumen_raw.txt", encoding='utf-8') as f:
    mediakonsumen_raw = f.readlines()
len(mediakonsumen_raw)

2241

Run cell below to see what the data looks like.

In [7]:
random.sample(mediakonsumen_raw, 5)

['Menanggapi keluhan tersebut, Layanan Pelanggan Home Credit sudah menghubungi Ibu Linda Supatmi untuk menindaklanjuti permasalahan ini dan memberikan penjelasan lebih lanjut. Perlu kami informasikan pula bahwa keluhan ini telah terselesaikan dengan baik.\n',
 'Agus Sulistyo\n',
 '081398729***\n',
 '0812822797**\n',
 'Jakarta Selatan 12520\n']

We can see some documents are identical! See if this is large enough to become a problem.

In [8]:
unique_document_size = len(list(set(mediakonsumen_raw)))
duplicated_document_size = len(mediakonsumen_raw) - unique_document_size
duplicate_ratio = duplicated_document_size / len(mediakonsumen_raw)
print(f"Unique documents size: {unique_document_size}")
print(f"Duplicated documents size: {duplicated_document_size}")
print("Duplicated documents ratio: {:.3f}".format(duplicate_ratio))

Unique documents size: 1349
Duplicated documents size: 892
Duplicated documents ratio: 0.398


Decide to remove the duplicated documents.

In [9]:
corpus = list(set(mediakonsumen_raw))
len(corpus)

1349

In [10]:
before_cleaning_stats = sparsity(corpus)

## Cleaning

This process must be done in order. Use the available RegExp collections, or create new one, then add them **one-by-one** while sampling the corpus to keep sanity check.

In [11]:
corpus = [s.lower() for s in corpus]
corpus = [regex_remove_url(s) for s in corpus]
corpus = [s.strip() for s in corpus]
corpus = [regex_simplify_punctuations(s) for s in corpus]
corpus = [regex_percentage_with_comma(s) for s in corpus]
corpus = [regex_currency(s) for s in corpus]
corpus = [regex_frequency(s) for s in corpus]
corpus = [regex_contract_number(s) for s in corpus]
corpus = [regex_phone_number(s) for s in corpus]
corpus = [regex_spell_number(s) for s in corpus]
corpus = [regex_atau(s) for s in corpus]
corpus = [regex_remove_non_charmap(s) for s in corpus]
corpus = [regex_multiple_punctuations(s) for s in corpus]
corpus = [regex_triplet_punctuations(s) for s in corpus]
corpus = [regex_remove_multiple_spaces(s) for s in corpus]
corpus = [s.strip() for s in corpus if s != '']

In [12]:
random.sample(corpus, 5)

['kebagusan, jakarta selatan dua belas ribu lima ratus dua puluh',
 'sehubungan dengan dimuatnya surat pembaca di portal berita mediakonsumen. com pada tanggal lima belas agustus dua ribu delapan belas yang dikirimkan oleh bapak arif nugraha yang berjudul sering mendapat telepon dari marketing home credit sangat mengganggu, maka dengan ini pt home credit indonesia menyampaikan permohonan maaf atas ketidaknyamanan yang dialami oleh pelanggan kami.',
 'pertama tama kami menyampaikan permohonan maaf atas ketidaknyamanan yang dialami oleh bapak meri irwan. layanan pelanggan home credit telah menindaklanjuti keluhan ini dengan menghubungi bapak meri irwan untuk memberikan penjelasan lebih lanjut dan mencarikan solusi untuk disepakati bersama.',
 'disini saya mau menceritakan masalah ketidaknyamanan dengan solusi home credit. pada bulan april dua ribu delapan belas teman saya mengambil hp, tapi karena alasan bi checking nya tidak bisa dipakai, dipinjamlah nama saya. lha sewaktu itu saya masi

## Compare Stats

In [13]:
after_cleaning_stats = sparsity(corpus)

In [14]:
print_stats(before_cleaning_stats)
print()
print_stats(after_cleaning_stats)

Number of documents: 1349
Sum of Ni: 34872
Vocab size: 5874
Sparsity: 0.9955992067668632

Number of documents: 1347
Sum of Ni: 36854
Vocab size: 4360
Sparsity: 0.9937247570203579


## Save Corpus and Vocab

In [15]:
with open("../../dataset/corpus/mediakonsumen_corpus.txt", 'w') as f:
    for document in corpus:
        f.write(f"{document}\n")

In [16]:
with open("../../dataset/corpus/mediakonsumen_vocab.txt", 'w') as f:
    for vocab in after_cleaning_stats['V']:
        vocab_clean = re.sub('[.,]', '', vocab)
        f.write(f"{vocab_clean}\n")

# Instagram Comment Corpus

In [17]:
instagram_df = pd.read_csv("../../dataset/corpus/raw/homecreditid_comments.csv")
instagram_df.shape

(26350, 4)

We only need those conversations in comment sections.

In [18]:
corpus = instagram_df['owner_comment'].values.tolist()

Run cell below to see what the comments looks like.

In [67]:
random.sample(corpus, 5)

['https://www.youtube.com/channel/UCvxef6NxP6D_1NBK8hBUAmQ https://youtu.be/ZZuwVpG0ICM https://youtu.be/-OcGPizXO8E subcribe ya',
 'Minn udah di umumin blm ini pemenang nya???',
 'Kolak, Biji Salak, Es Buah, Gorengan, Sop',
 'Ez first try😎',
 'Saya berhasil capture']

Save corpus statistics before cleaning.

In [19]:
before_cleaning_stats = sparsity(corpus)

## Cleaning

This processes must be done in order.

### Lower Casing

In [20]:
corpus = [s.lower() for s in corpus]

In [21]:
random.sample(corpus, 5)

['apakah bisa hci melampaui cicilan rumah gitu? karena saya salah satu client yg merasa diuntungkan dan diberikan kemudahan dalam hal cicilan, terimakasih semoga suatu saat hci makin berkembang ke ranah perumahan2 gitu, hehe suskes selalu ya!',
 'komitmen cerdas ala saya adalah selalu mengatur keuangan dengan bijak dan seperlunya saja,misalkan pengen jajan atau blanja bisa kok kita cari yg gak mahal" dan pastinya yang diskonan dong 😁 selain itu hindari juga jalan" setiap minggunya karena itu pemicu pemborosan lho kalau kita perlu/butuh apa" mending kita belinya di @homecreditid ajah karena bisa banget tuh punya barang impian tapi gak kluar duit banyak,bisa dong dicicil perbulannya ringan banget deh pkoknya,jadi uangnya bisa cukup tuh sampe sbulan gajian berikutnya,hidup jadi lebih teratur. @srirezeki5693 @hastuti_erni @ettyzalfa_zulfa #doitcerdas #yangkamumau',
 'yeayyy setelah 10 kali coba',
 '@jkt48shani',
 'raja ampat 😍']

### Remove @username

In [22]:
corpus = [regex_remove_username(s) for s in corpus]

In [23]:
random.sample(corpus, 5)

['tolong pengertiannya dari pihak hci saya sangat terganggu dengan tlp dari hci yang masuk bahkan sehari saya bisa di tlp 5x dan tidak tahu waktu. saya sudah beberapa kalo menolak penawaran yg ditawarkan tp kenapa masih di tlp juga dan berulang kali setiap harinya. saya jadi kesulitan untuk mengangkat tlp masuk dari nomer penting lainnya yg tidak diketahui',
 'dampak virus corona penghasilan  turun deratis angsuran tetap berjalan 😢',
 'e',
 'bernafas dalam air',
 '... •ketemu setan digang sempit •banyak penghuninya sudah dari dulu •kalau mau beli barang dengan sistem kredit •hanya  selalu berikan #yangkamumau #giveawayhomecredit ikutan juga yukk!! _sammypoerba  ']

### Remove Hashtag

In [24]:
corpus = [regex_remove_hashtag(s) for s in corpus]

In [25]:
random.sample(corpus, 5)

['kak 😁 convert pulsa bisa dijadikan apa saja ya? transfer bank? topup ovo, gopay, paypro? pasti bisa donk, atau ada keperluan lain untuk bayar ini itu??  ada solusinya, langsung hubungin kami saja ka : line :  wa : 0831-8387-5873 salam kami ',
 'min ada head unit double din',
 'dari pada buat giveaway kan mending urus komplain dulu gan  ini gk ada pertanggung jawabanya sama sekali homecredit, ati2 guys kalo kalian pake jasa disini',
 '👍👍',
 'stoooppp !!! apa2an ini !!!??? ga boleh sembarangan menuduh kalau ga ada buktinya !!! udah ga usah ikut2an. dosa tau !!!']

### Remove Url

In [26]:
corpus = [regex_remove_url(s) for s in corpus]

In [27]:
random.sample(corpus, 5)

['  ',
 'waspadalah waspadalah',
 '1. warna hijab wanita 2. sepatu pria 3.logo di dada yang pria 4.warna celana wanita 5. warna celana pria    ',
 'rumanh mantan anjirr',
 'halo admin,ini maksudnya dengan upload foto dan caption atau cuma repost banner dan caption ?']

### Simplify Punctuations

1. `?`, `!` --> `.`
2. `;` --> `,`
3. `'` --> `<empty>`

In [28]:
corpus = [''.join([simplified_punctuation_map.get(c, c) for c in s]) for s in corpus]

In [29]:
random.sample(corpus, 5)

['getjanuary gak ada min.',
 '5 dongg',
 'mau credit dana bisa 🙏🙏🙏',
 'baca dm ya min plis',
 'pelaku pencuri itu orang c, yang pake helm dan jaket merah. dah itu, aja seperti dah cukup penjelasan😮 _ap .octas _rizky_ramadhani']

### Remove Non Alphanumerical + Punctuation Character

This is done according to the character map used in the model.

In [30]:
corpus = [regex_remove_non_charmap(s) for s in corpus]

In [31]:
random.sample(corpus, 5)

['kalo di daerah bekasi dimananya storenya ,....',
 'sudah jauh datang buat bertemu dg agennyaa hasilnyaa malah zonk, agennya yg ditunggu   tunggu malah telat lagi ,gak tau apaa cuaca nya ituu hujan deras  pakaian pada basah semua, keringnya pakaian dg angin sajaa kesalahan d agennyaa pasti nih .terimakasih   sudah menolak sayaa.',
 'zonk berarti aku pdahl da byk x share',
 'management gaada majunya',
 'tolong respon dm saya min ']

### Percentage with Comma

In [32]:
corpus = [regex_percentage_with_comma(s) for s in corpus]

In [33]:
random.sample(corpus, 5)

['yuhuyyy done  ',
 'haloo min  tolong  bls dm saya.mau tnya klo mitra cilegon serang dimana yah',
 'aku udah ikutan nih... hehehe semoga aku dapet kamera, soalnya udah lama pengen banget punya kamera ',
 ' ngakak v',
 'min pengumuman getdesember.']

### Currency

In [34]:
corpus = [regex_currency(s) for s in corpus]

In [35]:
random.sample(corpus, 5)

['hati2 banyak penipuan mengatasnamakan homekredit',
 'semoga aku yang menang',
 'kasih info dong biar aku tdk di teror2 telpon pertama ketika diangkat mati terus telp nawarin2  terganggu tau nggak sih',
 '7 labu ',
 'ka anya tau ga perbedaannya ka anya sama cincin.. kalo cincin di jari manis kalo ka anya dihati aku eaaa      ']

### Frequency

In [36]:
corpus = [regex_frequency(s) for s in corpus]

In [37]:
random.sample(corpus, 5)

['mau ganti hp . yuk  aja sob ',
 'penyesalan seumur hidup kalo kredit di , siap siap di teror bahkan keluargapun ikut di teror. blokir aja homecredit. coba sekali penyesalan seumur hidup.',
 'mendung, becek, gerimis, genangan, badai   ',
 'so usseles aplication, i have money , lu semua rijek data gue, data keluarga gue , cuma gara2 gak ke angkat kalian rijek seenaknya , woi kalian bisa kan hubungi sekitar 1 jam kemudian, lucu kalian',
 'harus ikuttttt']

### Contract Number

In [38]:
corpus = [regex_contract_number(s) for s in corpus]

In [39]:
random.sample(corpus, 5)

['bagaimana klaim saya . saya sudah terlalu banyak dipersulit dari pihak homecredit, tolong dipercepat, jangan terus mengulur waktu, saya sangat kecewa dengan homecredit, terlalu ribet, sangat mempersulit pelanggan tolong segera dipercepat klaim refund tiket saya',
 'masa screenshotnya kuburan coba  oke gua cerita tentang kuburan, kuburan adalah tempat dimana kehidupan sesungguhnya baru dimulai siap tidak siap kita semua akan pergi kesana jadi apalah yang harus kita banggakan semua hanya milik allah yang maha pencipta tidak perlu kita sombongkan semua yang kita punya karna allah berfirman alangkah dahsyatnya sekiranya kamu melihat di waktu orang orang yang zalim berada dalam tekanan sakratul maut, sedang para malaikat memukul dengan tangannya, sambil berkata keluarkanlah nyawamu di hari ini kamu dibalas dengansiksa yang sangat menghinakan, karena kamu selalu mengatakan terhadap allah perkataan yang tidak benar dan karena kamu selalu menyombongkan diri terhadap ayat ayatnya. qs. al anam

### Phone Number

In [40]:
corpus = [regex_phone_number(s) for s in corpus]

In [41]:
random.sample(corpus, 5)

['no 4',
 'video 1 . cintol  serangga, yang kamu lakukan ke saya itu ........  cintol berpikir sangat lama untuk kelanjutannya. sampai sampai dia memesan kopi dulu .  serangga  hey penulis, lanjutkan ceritanya. apa kau tidak mau hadiah. kata serangga kepada penulis, karena sudah lama menunggu penulis  iya iya, lebih baik kalian berdua doakan aku mendapatkan lanjutannya, dan jangan lupa juga, doakan aku mendapatkan hadiah  kata penulisnya, mengajak mereka. c  s  ashiiiaap dengan suara khass . dan saat jalan, tiba tiba penulis mendapat ide . penulis  apa ku tulis jaaaaaaaaaaangkrik saja ya. tapi jaa nya panjang. supaya pembaca kira jahat padahal jangkrik . berbicara dengan hatinya dann. hingga saat ini, kira kira masih itu kelanjutannya .  video 2 . dahlan  kamu cantik. tapi aku belum mencintaimu, tapi gatau kalau.... kamu lepaskan kakiku yang kamu injek  kata dahlan dengan membisikkannya ke hidung mileak. mileak  emang aku injek. enggaa... mileak hampir pingsan karena bau mulut dahlan .

### Spell Number

In [42]:
corpus = [regex_spell_number(s) for s in corpus]

In [43]:
random.sample(corpus, 5)

['  jawaban aku  enam  kenapa harus saya yang di pilih karna saya mau hadiahin hp baru mama saya , biar bisa vidio call karna terpisah jarak , smoga rekeki aku ya min karna saya yakin dengan  dr give away nya saya pasti bisa ..  yuk ikutan dhea    yakin     ',
 'ada peniti di kapal selam  ada kue pancong ditengahnya bolong hati hati jalan malam  dibelakangmu ada pocong dan sundel bolong  panda seratus delapan puluh dua    ',
 'r',
 'itu yg dipake mas nya hp samsung apa ya .',
 'dfggggffjso']

### Trim Multiple Spaces

In [44]:
corpus = [' '.join(s.split()) for s in corpus]

In [45]:
random.sample(corpus, 5)

['yang c',
 'pencapaian terbesar saya di th dua ribu sembilan belas ialah berjodoh dengan home credit indonesia d karena di home credit aku bisa cicil barang impian yang aku mau tanpa kartu kredit, lebih enaknya lagi di home credit ada , jadi ngga pusing deh mikirin bunga wkwk di home credit ada juga asuransi untuk handphone yang kita pilih, jadi semisal ada kerusakan fisik secara tidak disengaja bisa diganti baru lohhh selain itu, bayar cicilan per bulan juga tanpa biaya admin di payment point, ngga kaya pembiayaan lain wkwk terimakasih home credit sudah percaya sama aku, terima juga buat mas sales yang item manis ramah yang udah bantu proses pengajuan cicilanku, prosesnya ngga sampai berhari hari, ngga sampai berjam jam, cuma dua puluh menit, udah di disetujui sama home credit hayooo teman teman seantero jagad netizenesia, silahkan beli dengan dari',
 'sembilan',
 'ada sembilan',
 'hai kalau menurut penerawangan saya sih yang mencuri laptop korban adalah orang a. kenapa orang a. yang

### Remove Multiple Punctuations

In [46]:
corpus = [regex_multiple_punctuations(s) for s in corpus]

In [47]:
random.sample(corpus, 5)

['sering foto dipesawat, kelilipan abu rokok di post satpam',
 'ku mau pinjam uang lg tapi angsuran tingal satu bulan lg bisa gk ngambil lagi',
 'warna celana perempuan, warna dan model sepatu cowo, saku pada baju cowo , warna jilbab, warna celana cowo, lambang homecredit di sblh kanan',
 'ribet disini harus punya npwp. bpjs ketenagakerjaan. klo yg gk punya gk bs kredit',
 'susah cairnya, pdhal ak tuh gk punya masalah bi ceking, dan tdk punya utang di lesing lain, buktinya gk di acc, pengajuan lima belas juta gk acc, katanya klu mau di acc pengajuan lima juta bisa di acc, prettt ternyata data udh masuk ehh tau tau lima juta juga gk di acc. klu niat mau bantu org bik yo jgn terlalu ribet, omongny aja empuk kyk telo']

### (Optional) Triplet Punctuactions

If using subword model, this step is optional, since space is treated as a basic symbol.

In [48]:
corpus = [regex_triplet_punctuations(s) for s in corpus]

In [49]:
random.sample(corpus, 5)

['jawabannya enam semoga kali ini saya bisa jadi pemenang nya, semoga saat ini ada rezeki buat saya karena pengen punya hape baru dan sudah sering ikutan giveaway di tapi belum pernah dapet dan semoga semakin sukses yukkk. andriastuty ikutan giveaway nya',
 'gerimis, mendung, becek, genengan, badai',
 'tiga',
 'bales lah kalo ada yang tanya tanya di dm, admin sudah selayaknya bales pertanyaan customer. kami bayar cicilan itu lancarin usaha kalian juga loh, lo kalo telat bayar kepo bgt kan. ',
 'ada promo cicilan nol persen gk buat hp samsung. ']

### Remove Empty String and Strip

In [50]:
corpus = [s.strip() for s in corpus if s != '']

In [51]:
random.sample(corpus, 5)

['bully aja bully hahahaha',
 'saya udah punya kontrakpembiyaan multiguna tpi bingung harus kemna ya supaya di acc',
 'ada tujuh. simbol atas kanan. payung. pohon di kanan. gedung di kanan. sepatu. jenggot. simbol kanan bawah.',
 'nenekku lagi rehat adikku pakek gigi kawat baju putih nampak terlihat ternyata hantu yang telah lewat nuriawaty dua puluh dua indra',
 'hai bioskop. ada kenangan tersendiri tiga belas tahun yang lalu di sana, saat aku mengenal mantan pacar yang sekarang udah jadi ayah dari anak anak ku, dan setiap tanggal sepuluh juli kami wajib nonton ke bioskop, bawa anak dua juga karena di bioskop lah awal aku dan dia bertemu setiawan sembilan puluh delapan ncha']

### Remove One Word Only Documents

In [52]:
corpus = [s for s in corpus if len(s.split()) > 1]

In [53]:
random.sample(corpus, 5)

['rumah mantan, yess gagal move on, bakalan gue ulang dari awal mulai dari kenalan lebih akrab lagi, dekat dengan keluarganya, makan bareng, ngobrol bareng, n jadian lagi yuk ikutan kerumah mantan gue fahmi seratus dua puluh tiga',
 'mimin mohon kesediaan membaca dm saya. trims mimin',
 'bismillah saya sudah ikutan',
 'satu pulang kampung dua angpao tiga baju baru empat tabungan lima investasi',
 'min kuis ini udah diumumin belom pemennagnya']

## Compare Stats

Recalculate stats after cleaning.

In [54]:
after_cleaning_stats = sparsity(corpus)

In [55]:
print_stats(before_cleaning_stats)

Number of documents: 26350
Sum of Ni: 550641
Vocab size: 82637
Sparsity: 0.9997471206402597


In [56]:
print_stats(after_cleaning_stats)

Number of documents: 22751
Sum of Ni: 511616
Vocab size: 42019
Sparsity: 0.9994648224156335


## Save Corpus and Vocab

In [57]:
with open("../../dataset/corpus/instagram_corpus.txt", 'w') as f:
    for document in corpus:
        f.write(f"{document}\n")

In [58]:
with open("../../dataset/corpus/instagram_vocab.txt", 'w') as f:
    for vocab in after_cleaning_stats['V']:
        vocab_clean = re.sub('[.,]', '', vocab)
        f.write(f"{vocab_clean}\n")

## (Optional) Sub Word Tokenizer

In case we want to use sub word token, or word pieces.

In [2]:
import sentencepiece as spm
spm.SentencePieceTrainer.train('--input=../../dataset/corpus/instagram_corpus.txt --model_prefix=m --vocab_size=8000')

True

In [3]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [18]:
print(sp.encode_as_pieces('homecredit kamu bisa'))
print(sp.encode_as_ids('homecredit kamu bisa'))

['▁homecredit', '▁kamu', '▁bisa']
[123, 72, 16]


In [20]:
print(sp.decode_pieces(['▁homecredit', '▁kamu', '▁bisa']))
print(sp.decode_ids([123, 72, 16]))

homecredit kamu bisa
homecredit kamu bisa


In [26]:
[sp.GetScore(id) for id in [123, 72, 16]]

[-6.7618937492370605, -6.207313537597656, -5.053555011749268]

In [127]:
for n in range(10):
    print(sp.sample_encode_as_pieces('homecredit kamu bisa', -1, 0.1))
for n in range(10):
    print(sp.sample_encode_as_ids('homecredit kamu bisa', -1, 0.1))

['▁', 'homecredit', '▁kamu', '▁bisa']
['▁ho', 'm', 'e', 'c', 'redit', '▁ka', 'mu', '▁bis', 'a']
['▁', 'homecredit', '▁kamu', '▁b', 'i', 's', 'a']
['▁', 'home', 'credit', '▁ka', 'mu', '▁', 'b', 'i', 's', 'a']
['▁', 'homecredit', '▁kam', 'u', '▁', 'bisa']
['▁', 'homecredit', '▁', 'ka', 'mu', '▁bi', 'sa']
['▁', 'h', 'o', 'm', 'e', 'c', 'r', 'e', 'd', 'it', '▁', 'k', 'a', 'mu', '▁b', 'i', 'sa']
['▁', 'homecredit', '▁ka', 'mu', '▁', 'b', 'isa']
['▁ho', 'me', 'c', 'redit', '▁k', 'am', 'u', '▁bisa']
['▁', 'home', 'c', 'r', 'e', 'd', 'i', 't', '▁', 'k', 'a', 'm', 'u', '▁bis', 'a']
[433, 2096, 106, 396, 5858, 11, 3504, 205, 16]
[11, 5089, 5101, 72, 16]
[433, 326, 1069, 5101, 131, 757, 205, 1031, 80]
[11, 5089, 5101, 11, 86, 80, 200, 205, 57, 42, 410]
[44, 396, 5858, 11, 86, 80, 219, 11, 2266]
[44, 5101, 72, 16]
[11, 94, 326, 1069, 5101, 4045, 205, 16]
[123, 72, 791, 410]
[1754, 200, 106, 5101, 72, 16]
[11, 5089, 5101, 72, 16]


In [128]:
# get 10 best
print(sp.nbest_encode_as_pieces('homecredit kamu bisa', 10))
print(sp.nbest_encode_as_ids('homecredit kamu bisa', 10))

[['▁homecredit', '▁kamu', '▁bisa'], ['▁', 'homecredit', '▁kamu', '▁bisa'], ['▁homecredit', '▁ka', 'mu', '▁bisa'], ['▁homecredit', '▁kamu', '▁', 'bisa'], ['▁homecredit', '▁kamu', '▁bis', 'a'], ['▁home', 'credit', '▁kamu', '▁bisa'], ['▁homecredit', '▁kamu', '▁b', 'isa'], ['▁homecredit', '▁kamu', '▁bi', 'sa'], ['▁homecredit', '▁kam', 'u', '▁bisa'], ['▁homecredit', '▁k', 'a', 'mu', '▁bisa']]
[[123, 72, 16], [11, 2390, 72, 16], [123, 380, 219, 16], [123, 72, 11, 2266], [123, 72, 1031, 80], [44, 5101, 72, 16], [123, 72, 57, 2752], [123, 72, 791, 410], [123, 4045, 205, 16], [123, 131, 80, 219, 16]]
